Below code will generate SV table for each gene.

In [10]:
import pandas as pd
from fuc import common

sv_genes = common.convert_file2list('data/sv-genes.list')
df = pd.read_csv('data/combined.csv')

sv_df = df[df.Gene.isin(sv_genes)]
sv_df = sv_df.pivot(index='Sample', columns='Gene', values='CNV')

mappings = {x+1: f'0{x+1}' if x < 9 else str(x+1) for x in range(10)}
batch_df = df[df.Gene == 'ABCB1'][['Sample', 'Batch']]
batch_df.Batch = batch_df.Batch.replace(mappings)
batch_dict = dict(zip(batch_df.Sample, batch_df.Batch))

base = 'https://raw.githubusercontent.com/sbslee/1kgp-pgx-paper/main/plot'

def one_sv(x, sample, batch):
    sv, gene = x.split(',')
    return f'[{sv}]({base}/{gene}/{batch}/{sample}_NYGC.png)'

def one_row(r):
    genes = r.index.to_series()
    r = r.str.cat(genes, sep=',')
    batch = batch_dict[r.name]
    r = r.apply(one_sv, args=(r.name, batch))
    return r
    
sv_df = sv_df.apply(one_row, axis=1)

for gene in sv_genes:
    with open(f'sv-tables/{gene}.md', 'w') as f:
        f.write(sv_df[gene].to_markdown(tablefmt='github'))